# Inception-v3

In [ ]:
%conda install -y gdown

In [ ]:
import zipfile
import gdown
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt

from pathlib import Path
from tensorflow.config import list_physical_devices
from tensorflow.keras.utils import plot_model, load_img, img_to_array
from tensorflow.keras.preprocessing import image_dataset_from_directory
from tensorflow.keras.applications.inception_v3 import InceptionV3
from tensorflow.keras.layers import Dropout, Dense
from tensorflow.keras import Model
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.losses import SparseCategoricalCrossentropy
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report

In [ ]:
zip_name = "dataset.zip"
wd = Path("/kaggle/working")
extract_path = Path(wd, "data")
model_path = Path(wd, "models/inception_v3.h5")
train_path = Path(extract_path, "train")
test_path = Path(extract_path, "test")

class_names = list()

## Model settings

In [ ]:
batch_size = 64
img_size = 299
epochs = 4
seed = 27
validation_split = 0.2

## Download dataset

In [ ]:
gdown.download(
    "https://drive.google.com/uc?id=1LOOzAQOIAXiWF7yjnGwaW3AQXKtvP1Eq",
    zip_name
)

zip_ref = zipfile.ZipFile(Path(wd, zip_name), 'r')
zip_ref.extractall(extract_path)
zip_ref.close()

## Set class names

In [ ]:
class_names = [
    'amanita',
    'boletus',
    'cantharellus',
    'craterellus',
    'macrolepiota',
    'morchella',
    'pleurotus',
    'psilocybe'
]

## Check number of GPUs

In [ ]:
print("Num GPUs Available: {}".format(len(list_physical_devices('GPU'))))

## Load training and validation data

In [ ]:
training_data = image_dataset_from_directory(
    directory=train_path,
    validation_split=validation_split,
    subset='training',
    labels='inferred',
    class_names=class_names,
    label_mode='int',
    batch_size=batch_size,
    image_size=(img_size, img_size),
    seed=seed,
    shuffle=True
)

validation_data = image_dataset_from_directory(
    directory=train_path,
    validation_split=validation_split,
    subset='validation',
    labels='inferred',
    class_names=class_names,
    label_mode='int',
    batch_size=batch_size,
    image_size=(img_size, img_size),
    seed=seed,
    shuffle=True
)

## Define model

In [ ]:
model = InceptionV3(
    include_top=False,
    weights='imagenet',
    input_shape=(img_size, img_size, 3),
    pooling='avg'
)

In [ ]:
top = Dense(units=2048, activation='relu')(model.output)
top = Dropout(rate = 0.2)(top)
top = Dense(units=len(class_names), activation='softmax')(top)

model = Model(
    inputs=model.input,
    outputs=top
)

optimizer = RMSprop(learning_rate=0.0001)

model.compile(
    optimizer=optimizer,
    loss=SparseCategoricalCrossentropy(),
    metrics=['accuracy']
)

## Plot model

In [ ]:
plot_model(
    model,
    to_file='inception_v3.png', 
    show_shapes=True,
    show_layer_names=True
)

## Train model

In [ ]:
history = model.fit(
    training_data,
    validation_data=validation_data,
    epochs=epochs,
    shuffle=True
)

## Save model

In [ ]:
model.save(model_path)

In [ ]:
def predict_and_evaluate(images, labels, class_names):   
    predictions = model.predict(np.array(images))
    predicted_labels = np.argmax(predictions, axis = 1)
    
    hits = 0

    for predicted_label, prediction, label in zip(predicted_labels, predictions, labels):        
        probability = np.max(prediction) * 100
        print(
            "{} with {:.2f}% probability (real class: {})".format(
                class_names[predicted_label],
                probability,
                class_names[label]
            )
        )
        
        hits = hits + 1 if label == predicted_label else hits
        
    accuracy = (hits / len(labels)) * 100
    return accuracy, predicted_labels

## Load test dataset

In [ ]:
test_data = list()
y_true = list()

for class_name in class_names:
    class_path = Path(test_path, class_name)
    for image_path in class_path.iterdir():
        image_data = load_img(
            image_path,
            target_size=(img_size, img_size),
            color_mode='rgb'
        )
        image_array = img_to_array(image_data)
        test_data.append(image_array)
        y_true.append(class_names.index(class_name))

## Get labels

## Predict

In [ ]:
accuracy, y_pred = predict_and_evaluate(test_data, y_true, class_names)
print("Accuracy is {:.2f}%".format(accuracy))

## Evaluate performance

In [ ]:
evaluation = model.evaluate(
    x=np.array(test_data),
    y=np.array(y_true),
    batch_size=batch_size
)
print("Loss is {:.4f}".format(evaluation[0]))
print("Accuracy is {:.2f}%".format(100 * evaluation[1]))

In [ ]:
print(classification_report(y_true, y_pred, target_names=class_names))

In [ ]:
matrix = confusion_matrix(y_true, y_pred)

sns.heatmap(
    matrix,
    square=True,
    annot=True,
    cbar=False,
    cmap=plt.cm.Blues,
    xticklabels=class_names,
    yticklabels=class_names
)

plt.xlabel('True Classes')
plt.ylabel('Predicted Classes')
plt.title('Confusion Matrix')
plt.savefig('confusion_matrix.png')
plt.show()